<a href="https://colab.research.google.com/github/Jualns/Neoplasia-Image-Classification/blob/main/Treinando_o_Modelo_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O que vai encontrar nesse Notebook

***

Aqui apresento o passo a passo de como treinei o modelo de visão computacional para prever onde estão os problemas nas imagens, dobra de tecido, objeto aleatório e vasos com células de sângue.

Utilizei o modelo pré-treinado da ultralytics YOLOv8 para fazer todo o processo, isso é seguir os seguintes passos:

<p align=""><a href="https://bit.ly/ultralytics_hub"><img width="1000" src="https://github.com/ultralytics/assets/raw/main/yolov8/banner-integrations.png"/></a></p>

Veja a documentação [YOLOv8 Train Docs](https://docs.ultralytics.com/modes/train/) para mais informações.



O processo de forma simplificado foi composto por:

1. Criar o dataset no [Roboflow](https://app.roboflow.com), do qual consiste em:
    * Subir todas as imagens ruins
    * Anotar manualmente todos os defeitos que encontrei
    * Gerar um conjunto de dados com as imagens e as marcações para que o YOLO consiga ser treinado
2. Conectar a uma das ferramentas de logging, [ClearMl](app.clear.ml/) ou [Comet](https://www.comet.com/jualns/general/view/new/panels)
3. Fazer o treinamento
4. Salvar o modelo para utilizar futuramente

# 1. Importando o Dataset com as Marcações
***

**Segue o código para importar seus dados:**

In [ ]:
#@title Pegando a Chave da API para o Roboflow (Oculto)
from google.colab import drive
drive.mount('/content/drive')

hidden_api_path = "/content/drive/MyDrive/Colab Notebooks/Hidden API/roboflow_api.txt"

with open(hidden_api_path, "r") as f:
    roboflow_api_key = f.read()

In [ ]:
!pip install roboflow

from roboflow import Roboflow

rf = Roboflow(api_key=roboflow_api_key)
project = rf.workspace("cidamo-group-ufpr").project("neoplasiaimageclassification")
dataset = project.version(1).download("yolov8")

!mkdir datasets # Criando a pasta que o YOLOv8 usa para ler os datasets
!mv NeoplasiaImageClassification-1/ datasets/. # Movendo meu dataset para a pasta criada

Para mim o arquivo "data.yaml" do meu dataset veio com alguns problemas pequenos, tive que fazer as seguintes alterações para o modelo executar:

**MOSTRAR COMO ALTERAR O YAML**

# 2. Configurações do ambiente da YOLO
***
Pip install `ultralytics` e [dependências](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt).

In [2]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.192 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/78.2 GB disk)


## Plataformas "Loggers"

São plataformas que vão registrar muitas informações úteis como:
- Log
- Melhor e Última versão do modelo treinado
- Estatísticas gerais e específicas
- Exemplo de imagens de treino e teste

Basicamente salvam tudo que você precisa para seguir com seu modelo. Sendo as duas que utilizei:


- [Comet](https://www.comet.com/jualns/general/view/new/panels)
- [ClearMl](app.clear.ml/)

Observação: É possível usar as duas plataformas ao mesmo tempo

In [ ]:
#@title Exemplo usando o Comet {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.9/561.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 43.6 MB/s eta 0:00:00
Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [3]:
#@title Exemplo usando o ClearML {run: 'auto'}
logger = 'ClearML' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

<IPython.core.display.Javascript object>


🤖 ClearML connected successfully - let's build something! 🚀


# 3. Treinando a IA com ados Personalizados
***

Treinando diretamente do terminal:

In [ ]:
# Treinar o YOLOv8n nos dados personalizados direto no terminal
!yolo train model=yolov8m.pt data=/content/datasets/NeoplasiaImageClassification-1/data.yaml epochs=300 imgsz=640

100% 49.7M/49.7M [00:00<00:00, 78.9MB/s]
Ultralytics YOLOv8.0.176 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/datasets/NeoplasiaImageClassification-1/data.yaml, epochs=300, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=No

### Treinando com código em Python

In [ ]:
from ultralytics import YOLO

# Lendo o modelo
#model = YOLO('yolov8n.yaml')  # Criando um modelo "vazio"
model = YOLO('yolov8m.pt')  # Lendo um modelo pré-treinado (recommended for training)

# Usando o modelo
results = model.train(data='/content/datasets/NeoplasiaImageClassification-1/data.yaml', epochs=3)  # Treinando o modelo
results = model.val()  # Avaliando a performance nos dados de Validação
results = model('/content/datasets/NeoplasiaImageClassification-1/test/images/1A_s0c0x10664-1600y2154-1200m11_jpg.rf.bbca9452feeb8b93fecf0adeb9843461.jpg')  # Prevendo uma imagem

# Etapa de exportação
results = model.export(format='onnx')  # Exportando para o formato ONNX

Ultralytics YOLOv8.0.178 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/datasets/NeoplasiaImageClassification-1/data.yaml, epochs=3, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=

ClearML Task: created new task id=0df35023edbe422bb9766a68c48b74ea


ClearML results page: https://app.clear.ml/projects/16c2bf414bbb48b9bdfed3fcb851eb55/experiments/0df35023edbe422bb9766a68c48b74ea/output/log


ClearML Initialized a new task. If you want to run remotely, please add clearml-init and connect your arguments before initializing YOLO.
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: fastai, sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jualns/general/f97f4c020ab842ef9be1e37a880b843e

TensorBoard: Start with 'tensorboard --logdir runs/detect/train4', view at http://localhost:6006/


2023-09-14 23:53:24,521 - clearml.Task - INFO - Storing jupyter notebook directly as code


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
train: Scanning /content/datasets/NeoplasiaImageClassification-1/train/labels.cache... 245 images, 1 backgrounds, 0 corrupt: 100%|██████████| 245/245 [00:00<?, ?it/s]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning /content/datasets/NeoplasiaImageClassification-1/valid/labels.cache... 57 images, 1 backgrounds, 0 corrupt: 100%|██████████| 57/57 [00:00<?, ?it/s]
Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 datal

2023-09-14 23:55:08,871 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/YOLOv8/colab.0df35023edbe422bb9766a68c48b74ea/models/best.pt


COMET INFO: Please wait for metadata to finish uploading (timeout is 3600 seconds)
Ultralytics YOLOv8.0.178 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients
val: Scanning /content/datasets/NeoplasiaImageClassification-1/valid/labels.cache... 57 images, 1 backgrounds, 0 corrupt: 100%|██████████| 57/57 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:09<00:00,  2.50s/it]
                   all         57        632      0.208      0.174      0.175     0.0736
             BloodCell         57        153          0          0    0.00129   0.000773
            TissueFold         57        479      0.417      0.349      0.348      0.146
Speed: 7.7ms preprocess, 36.5ms inference, 0.0ms loss, 14.1ms postprocess per image
Results saved to runs/detect/val

image 1/1 /content/datasets/NeoplasiaImageClassification-1/test/images/1A_s0c0

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 14.5s, saved as 'runs/detect/train4/weights/best.onnx' (98.8 MB)

Export complete (17.8s)
Results saved to /content/runs/detect/train4/weights
Predict:         yolo predict task=detect model=runs/detect/train4/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train4/weights/best.onnx imgsz=640 data=/content/datasets/NeoplasiaImageClassification-1/data.yaml  
Visualize:       https://netron.app


# 4. Exportando seu Modelo

Você pode exportar seu modelo baseado no YOLOv8 para os formatos abaixo apenas usando o argumento `format`, i.e. `format=onnx`. Entre no [YOLOv8 Export Docs](https://docs.ultralytics.com/modes/export/) para detalhes.

- 💡 ProTip: Export to [ONNX](https://onnx.ai/) or [OpenVINO](https://docs.openvino.ai/latest/index.html) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://developer.nvidia.com/tensorrt) for up to 5x GPU speedup.


| Format                                                             | `format` Argument | Model                     | Metadata | Arguments                                           |
|--------------------------------------------------------------------|-------------------|---------------------------|----------|-----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | ✅        | -                                                   |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | `torchscript`     | `yolov8n.torchscript`     | ✅        | `imgsz`, `optimize`                                 |
| [ONNX](https://onnx.ai/)                                           | `onnx`            | `yolov8n.onnx`            | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)             | `openvino`        | `yolov8n_openvino_model/` | ✅        | `imgsz`, `half`                                     |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | `engine`          | `yolov8n.engine`          | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` |
| [CoreML](https://github.com/apple/coremltools)                     | `coreml`          | `yolov8n.mlpackage`       | ✅        | `imgsz`, `half`, `int8`, `nms`                      |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | `saved_model`     | `yolov8n_saved_model/`    | ✅        | `imgsz`, `keras`                                    |
| [TF GraphDef](https://www.tensorflow.org/api_docs/python/tf/Graph) | `pb`              | `yolov8n.pb`              | ❌        | `imgsz`                                             |
| [TF Lite](https://www.tensorflow.org/lite)                         | `tflite`          | `yolov8n.tflite`          | ✅        | `imgsz`, `half`, `int8`                             |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | `edgetpu`         | `yolov8n_edgetpu.tflite`  | ✅        | `imgsz`                                             |
| [TF.js](https://www.tensorflow.org/js)                             | `tfjs`            | `yolov8n_web_model/`      | ✅        | `imgsz`                                             |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | `paddle`          | `yolov8n_paddle_model/`   | ✅        | `imgsz`                                             |
| [ncnn](https://github.com/Tencent/ncnn)                            | `ncnn`            | `yolov8n_ncnn_model/`     | ✅        | `imgsz`, `half`                                     |


### Exportando o modelo pelo terminal

In [ ]:
!yolo export model=yolov8m.pt format=torchscript

Ultralytics YOLOv8.0.176 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8m summary (fused): 218 layers, 25886080 parameters, 0 gradients

PyTorch: starting from 'yolov8m.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (49.7 MB)

TorchScript: starting export with torch 2.0.1+cu118...
TorchScript: export success ✅ 8.9s, saved as 'yolov8m.torchscript' (99.2 MB)

Export complete (13.1s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8m.torchscript imgsz=640  
Validate:        yolo val task=detect model=yolov8m.torchscript imgsz=640 data=coco.yaml  
Visualize:       https://netron.app
💡 Learn more at https://docs.ultralytics.com/modes/export


### Exportando o modelo com python

In [ ]:
results = model.export(format='onnx')  # Exportando para o formato ONNX

Assim o modelo fica salvo nos seus arquivos para ser utilizado futuramente.